## Creating aggregations

This chapter uses data gathered by the National UFO Reporting Center. The data is contained in the `Incidents` table and in this lesson, you will be aggregating values in the `DurationSeconds` column.

Instructions

1. Write a T-SQL query which will return the average, minimum, and maximum values of the `DurationSeconds` column.

In [ ]:
-- Calculate the average, minimum and maximum
SELECT AVG(DurationSeconds) AS Average, 
       MIN(DurationSeconds) AS Minimum, 
       MAX(DurationSeconds) AS Maximum
FROM Incidents;

# Average             Minimum               Maximum
# 5592.875782703282   0.07999999821186066   10526400

## Creating grouped aggregations

You can calculate statistics for each group using `GROUP BY`. For example, you can calculate the maximum value for each state using the following query:

```
SELECT State, 
       MAX(DurationSeconds)
FROM Incidents
GROUP BY State
```

To filter even further, for example, to find the values for states where the maximum value is greater than 10, you can use the `HAVING` clause.

Instructions

1. Write a T-SQL query to calculate the average, minimum, and maximum values of the `DurationSeconds` column grouped by `Shape`. _You need to select an additional column. What is it?_
2. Update the query to return only the records where the minimum of `DurationSeconds` column is greater than 1.

In [ ]:
-- Calculate the aggregations by Shape
SELECT Shape,
       AVG(DurationSeconds) AS Average, 
       MIN(DurationSeconds) AS Minimum, 
       MAX(DurationSeconds) AS Maximum
FROM Incidents
GROUP BY Shape;

# Shape     Average              Minimum   Maximum
# unknown   2053.769722814499    1         259200
# disk      1445.3135593220338   1         253800
# sphere    903.3034398034398    1         46800
# ...

In [ ]:
-- Calculate the aggregations by Shape
SELECT Shape,
       AVG(DurationSeconds) AS Average, 
       MIN(DurationSeconds) AS Minimum, 
       MAX(DurationSeconds) AS Maximum
FROM Incidents
GROUP BY Shape
-- Return records where minimum of DurationSeconds is greater than 1
HAVING MIN(DurationSeconds) > 1;

# Shape       Average              Minimum   Maximum
# chevron     1100.59375           2         21600
# rectangle   969.6132075471698    4         28800
# changing    3191.6744186046512   2         172800
# ...

## Removing missing values

There are a number of different techniques you can use to fix missing data in T-SQL and in this exercise, you will focus on returning rows with non-missing values. For example, to return all rows with non-missing `SHAPE` values, you can use:

```
SELECT *  
FROM Incidents
WHERE Shape IS NOT NULL
```

Instructions

1. Write a T-SQL query which returns only the `IncidentDateTime` and `IncidentState` columns where `IncidentState` is not missing.

In [ ]:
-- Return the specified columns
SELECT IncidentDateTime,
       IncidentState
FROM Incidents
-- Exclude all the missing values from IncidentState  
WHERE IncidentState IS NOT NULL;

# IncidentDateTime      IncidentState
# 2005-10-31 18:00:00   ny
# 2005-10-31 18:30:00   nj
# 2005-10-31 19:00:00   md
# ...

## Imputing missing values (I)

In the previous exercise, you looked at the non-missing values in the `IncidentState` column. But what if you want to replace the missing values with another value instead of omitting them? You can do this using the `ISNULL()` function. Here we replace all the missing values in the `Shape` column using the word `'Saucer'`:

```
SELECT  Shape, 
        ISNULL(Shape, 'Saucer') AS Shape2
FROM Incidents
```

You can also use `ISNULL()` to replace values from a different column instead of a specified word.

Instructions

1. Write a T-SQL query which only returns rows where `IncidentState` _is missing_.
2. Replace all the missing values in the `IncidentState` column with the values in the `City` column and name this new column `Location`.

In [ ]:
-- Check the IncidentState column for missing values and replace them with the City column
SELECT IncidentState, 
       ISNULL(IncidentState, City) AS Location
FROM Incidents
-- Filter to only return missing values from IncidentState
WHERE IncidentState IS NULL;

# IncidentState   Location
# null            aust
# null            ivan
# null            tuni
# ...

## Imputing missing values (II)

What if you want to replace missing values in one column with another and want to check the replacement column to make sure it doesn't have any missing values? To do that you need to use the `COALESCE` statement.

```
SELECT Shape, 
       City, 
       COALESCE(Shape, City, 'Unknown') as NewShape
FROM Incidents

Shape      City     NewShape
NULL       Orb      Orb
Triangle   Toledo   Triangle
NULL       NULL     Unknown

```

Instructions

1. Replace missing values in `Country` with the first non-missing value from `IncidentState` or `City`, in that order. Name the new column `Location`.

In [ ]:
-- Replace missing values 
SELECT Country, 
       COALESCE(Country, IncidentState, City) AS Location
FROM Incidents
WHERE Country IS NULL;

# Country   Location
# null      australia
# null      ivanka pri dunaji (slovakia)
# null      tunisia (in-flight; over ocean)
# ...

## Using CASE statements

In this exercise, you will use a `CASE` statement to create a new column which specifies whether the `Country` is USA or International.

Instructions

1. Create a new column, `SourceCountry`, defined from these cases:
    1. When `Country` is `'us'` then it takes the value `'USA'`.
    2. Otherwise it takes the value `'International'`.

In [ ]:
SELECT Country, 
CASE WHEN Country = 'us' THEN 'USA'
ELSE 'International'
END AS SourceCountry
FROM Incidents;

# Country   SourceCountry
# us        USA
# us        USA
# us        USA
# ...

## Creating several groups with CASE

In this exercise, you will write a `CASE` statement to group the values in the `DurationSeconds` into 5 groups based on the following ranges:

```
DurationSeconds        SecondGroup
<= 120                 1
> 120 and <= 600       2
> 600 and <= 1200      3
> 1201 and <= 5000     4
For all other values   5
```

Instructions

1. Create a new column, `SecondGroup`, that uses the values in the `DurationSeconds` column based on the ranges mentioned above.

In [ ]:
-- Complete the syntax for cutting the duration into different cases
SELECT DurationSeconds, 
-- Start with the 2 TSQL keywords, and after the condition a TSQL word and a value
CASE WHEN (DurationSeconds <= 120) THEN 1
-- The pattern repeats with the same keyword and after the condition the same word and next value
     WHEN (DurationSeconds > 120 AND DurationSeconds <= 600) THEN 2
-- Use the same syntax here
     WHEN (DurationSeconds > 601 AND DurationSeconds <= 1200) THEN 3
-- Use the same syntax here
     WHEN (DurationSeconds > 1201 AND DurationSeconds <= 5000) THEN 4
-- Specify a value
     ELSE 5 
     END AS SecondGroup   
FROM Incidents;

# DurationSeconds   SecondGroup
# 37800             5
# 5                 1
# 5                 1
# ...